In [27]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_diabetes
import matplotlib.pyplot as plt
import scipy.stats as scs

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
pd.set_option('display.max_columns', None)
df=pd.read_csv('dsi-regression-case-study/data/Train.csv')
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,PC120-6E,PC120,NaN,-6E,NaN,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators,NaN,EROPS w AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,S175,S175,NaN,NaN,NaN,NaN,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,EROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[['SalePrice','YearMade','MachineHoursCurrentMeter','UsageBand','saledate','ProductSize','state','ProductGroup','Enclosure']]
df.head()

,SalePrice,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,ProductSize,state,ProductGroup,Enclosure
0,66000,2004,68.0,Low,11/16/2006 0:00,NaN,Alabama,WL,EROPS w AC
1,57000,1996,4640.0,Low,3/26/2004 0:00,Medium,North Carolina,WL,EROPS w AC
2,10000,2001,2838.0,High,2/26/2004 0:00,NaN,New York,SSL,OROPS
3,38500,2001,3486.0,High,5/19/2011 0:00,Small,Texas,TEX,EROPS w AC
4,11000,2007,722.0,Medium,7/23/2009 0:00,NaN,New York,SSL,EROPS


In [4]:
mask = df.MachineHoursCurrentMeter == 0
df.loc[mask,'UsageBand'] = 'Low'

In [5]:
df.saledate = pd.DatetimeIndex(df['saledate']).year

In [6]:
#assign state to region
region = {
'Alabama': 'S',
'Alaska': 'W',
'Arizona': 'W',
'Arkansas': 'S',
'California': 'W',
'Colorado': 'W',
'Connecticut': 'NE',
'Delaware': 'S',
'Florida': 'S',
'Georgia': 'S',
'Hawaii': 'W',
'Idaho': 'W',
'Illinois': 'MW',
'Indiana': 'MW',
'Iowa': 'MW',
'Kansas': 'MW',
'Kentucky': 'S',
'Louisiana': 'S',
'Maine': 'NE',
'Maryland': 'S',
'Massachusetts': 'NE',
'Michigan': 'MW',
'Minnesota': 'MW',
'Mississippi': 'S',
'Missouri': 'MW',
'Montana': 'W',
'Nebraska': 'MW',
'Nevada': 'W',
'New Hampshire': 'NE',
'New Jersey': 'NE',
'New Mexico': 'W',
'New York': 'NE',
'North Carolina': 'S',
'North Dakota': 'MW',
'Ohio': 'MW',
'Oklahoma': 'S',
'Oregon': 'W',
'Pennsylvania': 'NE',
'Rhode Island': 'NE',
'South Carolina': 'S',
'South Dakota': 'MW',
'Tennessee': 'S',
'Texas': 'S',
'Utah': 'W',
'Vermont': 'NE',
'Virginia': 'S',
'Washington': 'W',
'Washington DC': 'S',
'West Virginia': 'S',
'Wisconsin': 'MW',
'Wyoming': 'W'    
}

df['Region'] = df.state.map(region)


In [7]:
df = df[['SalePrice', 'YearMade', 'UsageBand', 'saledate', 'ProductSize', 'ProductGroup', 'Enclosure', 'Region']]

In [8]:
df['BL'] = (df['ProductGroup'] == 'BL').astype(int)
df['MG'] = (df['ProductGroup'] == 'MG').astype(int)
df['SSL'] = (df['ProductGroup'] == 'SSL').astype(int)

In [9]:
df['LWL'] = ((df.ProductGroup =='WL') & (df.ProductSize=='Large')).astype(int)
df['CWL'] = ((df.ProductGroup =='WL') & (df.ProductSize=='Compact')).astype(int)
df['SWSL'] = ((df.ProductGroup =='WL') & (df.ProductSize=='Small')).astype(int)
df['NWL'] = ((df.ProductGroup =='WL') & (pd.isna(df.ProductSize))).astype(int)

In [10]:
df['LTTL'] = ((df.ProductGroup =='TTL') & (df.ProductSize=='Large')).astype(int)
df['MTTL'] = ((df.ProductGroup =='TTL') & (df.ProductSize=='Medium')).astype(int)
df['NTTL'] = ((df.ProductGroup =='TTL') & (pd.isna(df.ProductSize))).astype(int)

In [11]:
df['LMTEX'] = ((df.ProductGroup =='TEX') & (df.ProductSize=='Large / Medium')).astype(int)
df['MINTEX'] = ((df.ProductGroup =='TEX') & (df.ProductSize=='Mini')).astype(int)
df['STEX'] = ((df.ProductGroup =='TEX') & (df.ProductSize=='Small')).astype(int)
df['NTEX'] = ((df.ProductGroup =='TEX') & (pd.isna(df.ProductSize))).astype(int)

In [12]:
df.drop(columns=['ProductSize','ProductGroup'],inplace=True)

In [13]:
df['REGS']=(df['Region'] == 'S').astype(int)
df['REGNE']=(df['Region'] == 'NE').astype(int)
df['REGW']=(df['Region'] == 'W').astype(int)
df['REGMW']=(df['Region'] == 'MW').astype(int)
df['REGNA']=pd.isna(df.Region).astype(int)

In [14]:
df.drop(columns='Region', inplace=True)

In [15]:
df['LOW']=(df['UsageBand'] == 'Low').astype(int)
df['MED']=(df['UsageBand'] == 'Medium').astype(int)
df['HIGH']=(df['UsageBand'] == 'High').astype(int)
df['NANUSE']=pd.isna(df.UsageBand).astype(int)

In [17]:
df.drop(columns='UsageBand', inplace=True)

In [18]:
df.drop(columns='Enclosure', inplace=True)

In [21]:
df['Age']=df['saledate']-df['YearMade']
df.drop(columns=['saledate','YearMade'],inplace=True)

In [30]:
#Removes one term to prevent collinearity
df.drop(columns=['BL','NANUSE','REGNA'], inplace=True)

In [22]:
df.head()

,SalePrice,BL,MG,SSL,LWL,CWL,SWSL,NWL,LTTL,MTTL,NTTL,LMTEX,MINTEX,STEX,NTEX,REGS,REGNE,REGW,REGMW,REGNA,LOW,MED,HIGH,NANUSE,Age
0,66000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,2
1,57000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,8
2,10000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,3
3,38500,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,10
4,11000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,2


In [31]:
y = df.SalePrice
X=df.drop(columns=['SalePrice'])

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
scalar = StandardScaler().fit(X_train)
X_train = scalar.transform(X_train)
X_test = scalar.transform(X_test)

In [36]:
params = {'alpha': np.logspace(-1,5, 10)}
params

{'alpha': array([1.00000000e-01, 4.64158883e-01, 2.15443469e+00, 1.00000000e+01,
        4.64158883e+01, 2.15443469e+02, 1.00000000e+03, 4.64158883e+03,
        2.15443469e+04, 1.00000000e+05])}

In [37]:
gs = GridSearchCV(Ridge(), param_grid=params, cv=10, n_jobs=4)
gs.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'alpha': array([1.00000e-01, 4.64159e-01, 2.15443e+00, 1.00000e+01, 4.64159e+01,
       2.15443e+02, 1.00000e+03, 4.64159e+03, 2.15443e+04, 1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [38]:
r2_score(y_test,gs.predict(X_test))

0.2575828362493957